In [ ]:
import time
from tqdm import tqdm
import meep as mp
import math, cmath
import numpy as np 
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import shutil, os
from matplotlib import colors

### Simple Source Generator

In [ ]:
def phasor_to_png(phasor_arr, im_base):
    """ Converts steady-state phasors to png of time-average magnitudes """
    arr2 = im_base.to_rgba(np.absolute(phasor_arr))
    arr3 = (arr2 * 255).astype(np.int8) # convert to int8 (lossy conversion, rounded into buckets of 255)
    return arr3

In [ ]:
train_path = Path('./data')
train_path.mkdir(exist_ok=True)
(train_path/'x/').mkdir(exist_ok=True)
(train_path/'labels/').mkdir(exist_ok=True)
train_images = 2000

resolution = 10  # pixels/um

sxy = 10
dpml = 2
cell = mp.Vector3(sxy+2*dpml,sxy+2*dpml,0)
pml_layers = [mp.PML(dpml)]

freq = 0.66713 # 200 GHz
geometry = [mp.Block(mp.Vector3(x=mp.inf, y=mp.inf, z=mp.inf),
                     center=mp.Vector3(0, 0, 0),
                     material=mp.Medium(epsilon=1))]
src_cmpt = mp.Ez

phase_choices = np.array([1, 1j, -1, -1j])
phase_choices_r = np.array([0, 90, 180, 270])

def uniform_random_sources(n=None):
    if n is None: n = np.random.randint(2, 10)
    
    points = (sxy * np.random.random(size=(n, 2)) - (sxy/2.0)) * 0.95
    amplitudes = 0.1 + np.random.normal(size=(n), loc=0.9, scale=0.5)
    phase_indices = np.random.choice(4, size=(n))
    
    cmpl_phase = phase_choices[phase_indices]
    real_phase = phase_choices_r[phase_indices]
    ampl_phase = cmpl_phase * amplitudes
    
    cent_vec = [mp.Vector3(x=s[0], y=s[1], z=0) for s in points]
    
    sources = [mp.Source(src=mp.ContinuousSource(freq),
                         center=c, component=src_cmpt,
                         amplitude=amp) 
               for amp, c in zip(ampl_phase, cent_vec)]
    
    return sources, points, amplitudes, real_phase

max_meep_amplitude = 1.0
conversion_norm = colors.Normalize(vmin=0, vmax=max_meep_amplitude*376.7, clip=True)
norm = colors.Normalize(vmin=0, vmax=max_meep_amplitude, clip=True)

In [ ]:
elapsed = np.zeros(50)

for i in tqdm(range(elapsed.size), ncols=100): 
    sources, points, amplitudes, phases = uniform_random_sources()

    sim = mp.Simulation(cell_size=cell,
                        resolution=resolution,
                        geometry=geometry,
                        sources=sources,
                        boundary_layers=pml_layers,
                        force_complex_fields=True)
    sim.init_sim()
    
    st = time.time()
    sim.solve_cw()
    ed = time.time()
    
    elapsed[i] = ed - st


In [ ]:
np.mean(elapsed), np.std(elapsed)

(0.9346670722961425, 0.06467058758582732)

In [ ]:
def get_x_img(phasor_arr, points, amps, phases):
    x = np.zeros_like(phasor_arr, dtype=float)
    f = np.zeros_like(phasor_arr, dtype=float)
    
    xs = []
    ys = []
    for p in points:
        _fullwidth = 2 * dpml + sxy
        xs.append((x.shape[1]-1)*((p[1]+_fullwidth/2.0)/_fullwidth))
        ys.append((x.shape[0]-1)*((p[0]+_fullwidth/2.0)/_fullwidth))
    xs = np.round(xs).astype(int)
    ys = np.round(ys).astype(int)

    for xp, yp, a, ph in zip(xs, ys, amps, phases):
        x[yp, xp] = a
        f[yp, xp] = ph
        
    return x, f

In [ ]:
train_path = Path('./data_v2')
train_path.mkdir(exist_ok=True)
(train_path/'x/').mkdir(exist_ok=True)
(train_path/'labels/').mkdir(exist_ok=True)
train_images = 2000
show = None

for i in range(train_images):
    sources, points, amplitudes, phases = uniform_random_sources()

    sim = mp.Simulation(cell_size=cell,
                        resolution=resolution,
                        geometry=geometry,
                        sources=sources,
                        boundary_layers=pml_layers,
                        force_complex_fields=True)
    
    sim.init_sim() # takes 15s
    sim.solve_cw()

    arr = sim.get_array(component=mp.Ez)

    x_img, phase_img = get_x_img(arr, points, amplitudes, phases)
    
    if show == None: 
        fig, ax = plt.subplots(2, 1)
        ax[0].imshow(x_img, norm=norm)
        ax[1].imshow(np.abs(arr), norm=norm)
        
    np.save(train_path/'x'/'train_{:04d}'.format(i), np.stack([x_img, phase_img], axis=0))
    np.save(train_path/'labels'/'train_{:04d}_label'.format(i), np.abs(arr))

-----------
Initializing structure...
time for choose_chunkdivision = 0.000158261 s
Working in 2D dimensions.
Computational cell is 14 x 14 x 0 with resolution 10
     block, center = (0,0,0)
          size (1e+20,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
time for set_epsilon = 0.0425283 s
-----------
Meep: using complex fields.
final residual = 9.06698e-09
Finished solve_cw after 262 CG iters (~ 1048 timesteps).
-----------
Initializing structure...
time for choose_chunkdivision = 0.000167062 s
Working in 2D dimensions.
Computational cell is 14 x 14 x 0 with resolution 10
     block, center = (0,0,0)
          size (1e+20,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
time for set_epsilon = 0.0411171 s
-----------
Meep: using complex fields.
final residual = 9.72743e-09
Finished solve_cw after 244 CG iters (~ 976 timesteps).
-----------
Initializing stru

In [ ]:
!ls

checkpoints      data_v2          lightning_logs
data             generation.ipynb training.ipynb
